# How Can One Save an Instance of an Estimator?

I have a situation in which I want to use an estimator that has been fit in an original Notebook to predict new values given data in a new Notebook.  The question is, can I save a specific model instance to be read in elsewhere?  This Notebook will run a basic regression on simulated data.  We will then attempt to write the model to disk, read back in, and use it to predict outcomes on new data.

In [1]:
#Basic data manipulation
import numpy as np
import pandas as pd
from pandas import Series, DataFrame

#Estimating methods
import statsmodels.formula.api as smf
from statsmodels.api import add_constant

#Serialization
import pickle

## Simulation of Input Data

We are going to follow a very basic model:

$Y = X_1 + 2X_2 + \epsilon$

In [2]:
#Generate random vectors for regressors
x1,x2=(np.random.uniform(20,size=50),np.random.uniform(20,size=50))

#Generate dependent variable
y=x1 + 2*x2 + [np.random.uniform(3) for val in range(len(x1))]

#Capture data in DF
data=DataFrame({'y':y,
                'x1':x1,
                'x2':x2})

data

,x1,x2,y
0,12.617356,17.612900,49.682436
1,11.901858,18.435362,50.100384
2,12.525232,11.761927,38.199787
3,5.028415,10.622601,27.959130
4,3.967293,14.911156,36.131878
5,16.681216,18.785751,56.441159
6,3.940214,17.560113,40.547529
7,2.666995,5.879624,16.052696
8,15.868034,19.910271,58.411995
9,10.825121,8.960415,30.628391


## Estimation of the Model

We are just going to fit a standard OLS estimator...

In [3]:
#Fit model
model=smf.ols(formula='y ~ x1 + x2', data=data).fit()

model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.998
Model:                            OLS   Adj. R-squared:                  0.998
Method:                 Least Squares   F-statistic:                 1.132e+04
Date:                Thu, 07 Apr 2016   Prob (F-statistic):           8.51e-64
Time:                        16:45:14   Log-Likelihood:                -41.235
No. Observations:                  50   AIC:                             88.47
Df Residuals:                      47   BIC:                             94.21
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
Intercept      2.3271      0.225     10.339      0.000         1.874     2.780
x1             0.9937      0.015     68.213      0.000         0.964     1.023
x2             1.9769      0.014    138.193      0.000         1.948     2.006
==============================================================================
Omnibus:                        5.728   Durbin-Watson:                   2.808
Prob(Omnibus):                  0.057   Jarque-Bera (JB):                2.211
Skew:                           0.087   Prob(JB):                        0.331
Kurtosis:                       1.985   Cond. No.                         42.4
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

To provide a point of comparison, we will use the regressor subset to predict `y`.

In [4]:
pred_y=model.predict(data[['x1','x2']])

pred_y

array([ 49.68324849,  50.59819518,  38.02499531,  28.32334071,
        35.74692776,  56.0399629 ,  40.95670828,  16.60054664,
        57.45497993,  30.79737997,  31.09070682,  29.01646067,
        30.70771374,  15.13976515,  32.69122126,  52.88250679,
        40.47138326,  38.41338352,  40.02640176,  45.86731056,
        25.7615701 ,  39.2536628 ,  39.9903827 ,  25.52275544,
        25.93684064,  16.76998397,  21.2331405 ,  28.66915131,
        31.96669684,  27.3512382 ,  31.84041738,  20.75951161,
        12.67331004,  17.85915957,  20.91362819,  23.0785252 ,
         5.56399047,  19.30926417,  45.16361241,  22.38135163,
        26.71722121,  32.6752277 ,  42.73414915,  25.61154646,
        37.56248332,  22.50262251,  52.46293217,  37.31877264,
         9.90775458,  21.78270546])

## Writing the Model to Disk

Now that we have instantiated this model object, let's see if we can write it to disk.  To do this, we are going to use a method called "pickling", which serializes an object into a byte stream so that it may be written to disk.  In theory, we can read this same byte stream back elsewhere, and use it to reconstitute the object that had been written to disk.

So, what exactly are we writing to disk?  Here is the ASCII version of the byte stream:

In [5]:
pickle.dumps(model)

'ccopy_reg\n_reconstructor\np0\n(cstatsmodels.regression.linear_model\nRegressionResultsWrapper\np1\nc__builtin__\nobject\np2\nNtp3\nRp4\n(dp5\nS\'__doc__\'\np6\nS\'\\n    Results class for for an OLS model.\\n\\n    Most of the methods and attributes are inherited from RegressionResults.\\n    The special methods that are only available for OLS are:\\n\\n    - get_influence\\n    - outlier_test\\n    - el_test\\n    - conf_int_el\\n\\n    See Also\\n    --------\\n    RegressionResults\\n\\n    \'\np7\nsS\'_results\'\np8\ng0\n(cstatsmodels.regression.linear_model\nOLSResults\np9\ng2\nNtp10\nRp11\n(dp12\nS\'normalized_cov_params\'\np13\ncnumpy.core.multiarray\n_reconstruct\np14\n(cnumpy\nndarray\np15\n(I0\ntp16\nS\'b\'\np17\ntp18\nRp19\n(I1\n(L3L\nL3L\ntp20\ncnumpy\ndtype\np21\n(S\'f8\'\np22\nI0\nI1\ntp23\nRp24\n(I3\nS\'<\'\np25\nNNNI-1\nI-1\nI0\ntp26\nbI00\nS\'\\xf6\\x03\\x7fQW\\x01\\xc4?z\\xe0\\x18P\\xfd\\xa1}\\xbf\\xe2>*(\\xf48z\\xbfz\\xe0\\x18P\\xfd\\xa1}\\xbf^\\xc7\\xc0\\xf9\\xd1s

I guess I am just trusting that this is correct.  Let's write that nonsense to disk.

In [6]:
with open('model_dump','w') as f:
    pickle.dump(model,f)

Now we can read the model object back in, save it as a new instance, and use it to predict based upon the original regressor data.  This result can be compared to the original prediction above.

In [7]:
#Capture model object from disk
with open('model_dump','r') as f:
    model2=pickle.load(f)

#Show model summary
print model2.summary()

#Predict new values of y
# pred_y2=model2.predict(data[['x1','x2']])

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.998
Model:                            OLS   Adj. R-squared:                  0.998
Method:                 Least Squares   F-statistic:                 1.132e+04
Date:                Thu, 07 Apr 2016   Prob (F-statistic):           8.51e-64
Time:                        16:45:14   Log-Likelihood:                -41.235
No. Observations:                  50   AIC:                             88.47
Df Residuals:                      47   BIC:                             94.21
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
Intercept      2.3271      0.225     10.339      0.0

    ---------------------------------------------------------------------------
    AttributeError                            Traceback (most recent call last)
    <ipython-input-16-f7a79b10e2b5> in <module>()
          7 
          8 #Predict new values of y
    ----> 9 pred_y2=model2.predict(data[['x1','x2']])

    C:\Users\marvinw\AppData\Local\Continuum\Anaconda\lib\site-packages\statsmodels\base\model.pyc in predict(self, exog, transform, *args, **kwargs)
        876         if transform and hasattr(self.model, 'formula') and exog is not None:
        877             from patsy import dmatrix
    --> 878             exog = dmatrix(self.model.data.orig_exog.design_info.builder,
        879                     exog)
        880         return self.model.predict(self.params, exog, *args, **kwargs)

    C:\Users\marvinw\AppData\Local\Continuum\Anaconda\lib\site-packages\pandas\core\generic.pyc in __getattr__(self, name)
       1976                 return self[name]
       1977             raise AttributeError("'%s' object has no attribute '%s'" %
    -> 1978                                  (type(self).__name__, name))
       1979 
       1980     def __setattr__(self, name, value):

    AttributeError: 'DataFrame' object has no attribute 'design_info'

Hmmmm.  While it does appear that we have reconstituted the object, clearly something was lost in translation.  This helpful [SO post](http://stackoverflow.com/questions/20724919/pandas-dataframe-attributeerror-dataframe-object-has-no-attribute-design-inf) reveals that user additions to the model can get lost.  In this case, `design_info` is referring to the formula used to fit the model: `'y ~ x1 + x2'`.  The post suggests either building a design matrix explicitly with **`patsy`**, or just making sure we add a constant to our input data for the unpickled version of the model.

In [8]:
#Predict new values of y
pred_y2=model2.predict(add_constant(data[['x1','x2']]),transform=False)

pred_y-pred_y2

array([  0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
         0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
         0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
         0.00000000e+00,  -3.55271368e-15,   0.00000000e+00,
         0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
         0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
         0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
         0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
         3.55271368e-15,   3.55271368e-15,   0.00000000e+00,
        -3.55271368e-15,   0.00000000e+00,   0.00000000e+00,
         3.55271368e-15,  -3.55271368e-15,   0.00000000e+00,
         0.00000000e+00,   3.55271368e-15,   3.55271368e-15,
         0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
         0.00000000e+00,   3.55271368e-15,   0.00000000e+00,
         0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
         0.00000000e+00,   0.00000000e+00,  -7.10542736e-15,
         0.00000000e+00,

I think that I can handle those small deviations.  Looks like this works as expected.

In [9]:
np.random.choice(range(0,2),size=10)

array([0, 1, 0, 0, 1, 1, 1, 0, 1, 1])

In [10]:
import statsmodels.api as sm
X=data[['x1','x2']].values
y=np.random.choice(range(0,2),size=len(data))
p=sm.Probit(y,X).fit(maxiter=1000)
p

Optimization terminated successfully.
         Current function value: 0.678036
         Iterations 4


In [11]:
# summary_string=p.summary().as_csv().split(',')
# ll_text_idx=[i for i,content in enumerate(summary_string) if 'Log-Likelihood:' in content][0]
# ll_rep_value=summary_string[conv_text_idx+1].strip().split('\n')[0]
# ll_max_found=ll_rep_value != 'nan'
# ll_max_found

In [12]:
p.summary().as_csv().split(',')

['                  Probit Regression Results                   \nDep. Variable:',
 'y               ',
 '  No. Observations:  ',
 '    50  \nModel:        ',
 'Probit          ',
 '  Df Residuals:      ',
 '    48  \nMethod:       ',
 'MLE             ',
 '  Df Model:          ',
 '     1  \nDate:         ',
 'Thu',
 ' 07 Apr 2016',
 '  Pseudo R-squ.:     ',
 '0.02180 \nTime:         ',
 '16:45:15        ',
 '  Log-Likelihood:    ',
 ' -33.902\nconverged:    ',
 'True            ',
 '  LL-Null:           ',
 ' -34.657\n              ',
 '                ',
 '  LLR p-value:       ',
 '0.2190  \n  ',
 '   coef   ',
 ' std err ',
 '    z    ',
 'P>|z| ',
 ' [95.0% Conf. Int.]\nx1',
 '    0.0201',
 '    0.023',
 '    0.884',
 ' 0.377',
 '   -0.024     0.065\nx2',
 '   -0.0298',
 '    0.024',
 '   -1.223',
 ' 0.222',
 '   -0.078     0.018']

In [13]:
print p.summary().tables[0]

                          Probit Regression Results                           
Dep. Variable:                      y   No. Observations:                   50
Model:                         Probit   Df Residuals:                       48
Method:                           MLE   Df Model:                            1
Date:                Thu, 07 Apr 2016   Pseudo R-squ.:                 0.02180
Time:                        16:45:15   Log-Likelihood:                -33.902
converged:                       True   LL-Null:                       -34.657
                                        LLR p-value:                    0.2190


In [14]:
dir(p)

['__class__',
 '__delattr__',
 '__dict__',
 '__doc__',
 '__format__',
 '__getattribute__',
 '__getstate__',
 '__hash__',
 '__init__',
 '__module__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_cache',
 '_data_attr',
 '_get_endog_name',
 '_get_robustcov_results',
 'aic',
 'bic',
 'bse',
 'conf_int',
 'cov_kwds',
 'cov_params',
 'cov_type',
 'df_model',
 'df_resid',
 'f_test',
 'fittedvalues',
 'get_margeff',
 'initialize',
 'k_constant',
 'llf',
 'llnull',
 'llr',
 'llr_pvalue',
 'load',
 'mle_retvals',
 'mle_settings',
 'model',
 'nobs',
 'normalized_cov_params',
 'params',
 'pred_table',
 'predict',
 'prsquared',
 'pvalues',
 'remove_data',
 'resid_dev',
 'resid_generalized',
 'resid_pearson',
 'resid_response',
 'save',
 'scale',
 'summary',
 'summary2',
 't_test',
 'tvalues',
 'use_t',
 'wald_test']

In [15]:
p.llf

-33.901802318125952

In [16]:
p.mle_retvals['converged']

True

In [17]:
p.pvalues

array([ 0.37650962,  0.22150798])

In [18]:
not np.isnan(p.pvalues).all()

True

In [19]:
p.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                          Probit Regression Results                           
==============================================================================
Dep. Variable:                      y   No. Observations:                   50
Model:                         Probit   Df Residuals:                       48
Method:                           MLE   Df Model:                            1
Date:                Thu, 07 Apr 2016   Pseudo R-squ.:                 0.02180
Time:                        16:45:15   Log-Likelihood:                -33.902
converged:                       True   LL-Null:                       -34.657
                                        LLR p-value:                    0.2190
==============================================================================
                 coef    std err          z      P>|z|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
x1             0.0201      0.023      0.884      0.377        -0.024     0.065
x2            -0.0298      0.024     -1.223      0.222        -0.078     0.018
==============================================================================
"""

In [27]:
p.summary().as_text()

'                          Probit Regression Results                           \n==============================================================================\nDep. Variable:                      y   No. Observations:                   50\nModel:                         Probit   Df Residuals:                       48\nMethod:                           MLE   Df Model:                            1\nDate:                Thu, 07 Apr 2016   Pseudo R-squ.:                 0.02180\nTime:                        17:18:45   Log-Likelihood:                -33.902\nconverged:                       True   LL-Null:                       -34.657\n                                        LLR p-value:                    0.2190\n==============================================================================\n                 coef    std err          z      P>|z|      [95.0% Conf. Int.]\n------------------------------------------------------------------------------\nx1             0.0201      0.023      0